# Calling Predict API On Model Hosted on Sagemaker

## Initialize Fiddler Client
We begin this section as usual by establishing a connection to our
Fiddler instance. We can establish this connection either by specifying 
our credentials directly, or by utilizing our `fiddler.ini` file. More
information can be found in the [setup](https://github.com/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/00%20Setup.ipynb) section.

In [ ]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

## Upload Dataset
To upload a model, you first need to upload a sample of the data of the model’s 
inputs, targets, and additional metadata that might be useful for model analysis. 
This data sample helps us (among other things) to infer the model schema and the 
data types and values range of each feature.

In [ ]:
import pandas as pd

features = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_range_low', 'total_acc']
target = 'loan_status'
dataset_id = 'p2p_loans'

if dataset_id in client.list_datasets():
    df = client.get_dataset(dataset_id)['test']
    df_schema = client.get_dataset_info(dataset_id)
else:
    df = pd.read_csv('/app/fiddler_samples/samples/datasets/p2p_loans/p2p_loans.csv')
    df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)
    upload_result = client.upload_dataset(
        dataset={'test': df}, 
        dataset_id=dataset_id,
        info=df_schema)
    
df_input = df[features]

## Generate and Write Model Info
As you must have noted, in the dataset upload step we did not ask for the model’s 
features and targets, or any model specific information. That’s because we 
allow for linking multiple models to a given dataset schema. Hence we require 
an Infer model schema step which helps us know the features relevant to the 
model and the model task. Here you can specify the input features, the target 
column, decision columns and metadata columns, and also the type of model.

We will save this information in a `model.yaml` file within our model directory.

In [ ]:
import pathlib
import shutil
import pickle
import yaml

project_id = 'tutorial'
model_id = 'sagemaker_hosted_model'

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=client.get_dataset_info(dataset_id),
    target=target, 
    features=features,
    display_name='SagemakerHostedModel',
    description='this model is hosted on Sagemaker.'
)

# create temp dir
model_dir = pathlib.Path(model_id)
shutil.rmtree(model_dir, ignore_errors=True)
model_dir.mkdir()

# save model schema
with open(model_dir / 'model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)

## Write `package.py`
A wrapper is needed between Fiddler and the model. This wrapper can be used to 
translate the inputs and outputs to fit what the model expects and what Fiddler 
is able to consume. More information can be found [here](https://docs.fiddler.ai/api-reference/package-py/)

In [ ]:
%%writefile sagemaker_hosted_model/package.py

import pickle
from pathlib import Path
import pandas as pd
import boto3
import random
from io import StringIO
import csv
import numpy as np


PACKAGE_PATH = Path(__file__).parent

class SklearnModelPackage:
    is_classifier = False
    output_columns = ['probability_Fully Paid']
    endpoint_name = 'sagemaker-xgboost-lending'
    content_type = "text/csv"                                        
    accept = "application/json"
    region = 'us-west-2'
    aws_access_key_id ='...'
    aws_secret_access_key='...'

    def __init__(self):
        self.client = boto3.client('sagemaker-runtime', region_name=self.region, 
                      aws_access_key_id=self.aws_access_key_id, 
                      aws_secret_access_key=self.aws_secret_access_key)
        
    def predict(self, input_df):
        payload = input_df.to_csv(index=False, header=False)
        return self.call_sagemaker(payload)

    def call_sagemaker(self, payload):
        custom_attributes = f'fiddler-sagemaker-id-{random.randint(0, 10000000)}'
        response = self.client.invoke_endpoint(
            EndpointName=self.endpoint_name, 
            CustomAttributes=custom_attributes, 
            ContentType=self.content_type,
            Accept=self.accept,
            Body=payload
        )
        csv_str = ''
        for row in response['Body']:
            csv_str += row.decode('utf-8')
            
        pd_array = pd.array(csv_str.split(','))
        pd_array = pd.to_numeric(pd_array, errors='coerce')
        return pd.DataFrame(pd_array, columns=self.output_columns)
    
def get_model():
    return SklearnModelPackage()



## Upload Model Package
Now that we have all the parts that we need, we can go ahead and upload the model to the Fiddler platform. You can use the [upload_model_package](https://docs.fiddler.ai/api-reference/python-package/#upload-model-package) to upload this entire directory in one shot. We need the following for uploading a model:
- The `path` to the model directory
- The `project_id` to which the model belongs
- The `model_id`, which is the name you want to give the model. You can access it in Fiddler henceforth via this ID
- The `dataset` which the model is linked to (optional)  

For this Sagemaker example, we will have a `model.yaml`, and a `package.py` file within our model directory.

In [ ]:
client.delete_model(project_id, model_id)
client.upload_model_package(model_dir, project_id, model_id)

## Test Model
Now, let's test out our model by interfacing with the client and 
calling [run model](https://docs.fiddler.ai/api-reference/python-package/#run-model).

In [ ]:
prediction_input = df_input[0: 10]
result = client.run_model(project_id, model_id, prediction_input)
result